In [1]:
import numpy as np
import pandas as pd
import rasterio
from PIL import Image
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
im = Image.open('../m_3009743_sw_14_060_20201018.tif')
im = np.array(im)

/Users/ziyanliu/opt/anaconda3/lib/python3.9/site-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (133302400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [3]:
im.shape

(12320, 10820, 4)

In [4]:
raster = rasterio.open('../m_3009743_sw_14_060_20201018.tif')
array = raster.read()

Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


In [5]:
array.shape

(4, 12320, 10820)

In [6]:
trial = im[42:50, 55:60, :]

In [7]:
# 5 x 5 window
def get_feature_arr(im):
    M, N, F = im.shape
    feature = np.zeros((M, N, (F + 1) * 5)) # (R, G, B, NIR, NDVI) * (1, min, max, mean, std)
    for i in range(M):
        for j in range(N):
            for k in range(4):
                feature[i, j, k] = im[i, j, k] # R, G, B, NIR
            if im[i, j, 3] + im[i, j, 0] == 0:
                ndvi = 0
            else:
                ndvi = (im[i, j, 3] - im[i, j, 0]) / (im[i, j, 3] + im[i, j, 0])
            feature[i, j, 4] = ndvi # NDVI
    padded = np.pad(feature, ((2, 2), (2, 2), (0, 0)), mode='edge')
   
    for r in range (M):
        for c in range(N):
            for w in range(5):
                window = padded[r:r+5, c:c+5, w]
                f_min = np.amin(window)
                f_max = np.amax(window)
                f_mean = np.mean(window)
                f_std = np.std(window)
                feature[r, c, w+5] = f_min
                feature[r, c, w+10] = f_max
                feature[r, c, w+15] = f_mean
                feature[r, c, w+20] = f_std
    return feature
            
        

In [8]:
feature = get_feature_arr(trial)


/var/folders/g5/z71zl0dx3vnbsmfqns6fm44w0000gn/T/ipykernel_36419/3131320919.py:9: RuntimeWarning: overflow encountered in ubyte_scalars
  if im[i, j, 3] + im[i, j, 0] == 0:
/var/folders/g5/z71zl0dx3vnbsmfqns6fm44w0000gn/T/ipykernel_36419/3131320919.py:12: RuntimeWarning: overflow encountered in ubyte_scalars
  ndvi = (im[i, j, 3] - im[i, j, 0]) / (im[i, j, 3] + im[i, j, 0])


In [9]:
print(feature[2, 2,])
print(trial[2,2,])

[1.35000000e+02 1.18000000e+02 1.15000000e+02 1.19000000e+02
 9.44881916e-01 1.16000000e+02 1.05000000e+02 9.80000000e+01
 7.70000000e+01 4.29184549e-03 2.05000000e+02 1.99000000e+02
 1.78000000e+02 2.00000000e+02 4.16666679e+01 1.56200000e+02
 1.50920000e+02 1.33560000e+02 1.48760000e+02 3.23625628e+00
 2.79685538e+01 2.79741595e+01 2.22028467e+01 3.52536863e+01
 8.12135314e+00]
[135 118 115 119]


In [ ]:
feature_sw = get_feature_arr(im)

In [ ]:
a, b, c = feature_sw.shape
f_2d = feature_sw.reshape(a*b, c)

In [ ]:
np.savetxt("austin_sw.csv", f_2d, delimiter=",")